## Import

In [95]:
import math
import pandas as pd
import numpy as np
import seaborn as sns

from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

from surprise import Reader, Dataset, SVD
from surprise import KNNBasic
from surprise import accuracy
from surprise import accuracy

from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

sns.set_style("darkgrid")

### Importare dataset

In [133]:
df1 = pd.read_csv('./combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])

df1['Rating'] = df1['Rating'].astype(float)

print('Dataset 1 shape:',df1.shape)

# Mi genera un nuovo indice da 1
df1 = df1.reset_index()
print('Dataset 1 shape without rating null:',df1.shape)

print('-Dataset examples-')
print(df1[:5])


Dataset 1 shape: (24058263, 2)
Dataset 1 shape without rating null: (24058263, 3)
-Dataset examples-
   index  Cust_Id  Rating
0      0       1:     NaN
1      1  1488844     3.0
2      2   822109     5.0
3      3   885013     4.0
4      4    30878     4.0


### Prendo i primi x , così che il calcolo del film non sia lunghissimo ma genero tante colonne di film

In [134]:
#df1 = df1.head(5000000)
df1 = df1.head(10000)

### df_nan è un DataFrame che indica True per le righe in cui la colonna 'Rating' è nulla e False altrimenti. Quindi, ogni volta che c'è una transizione da False a True nella colonna 'Rating', rappresenta l'inizio di un nuovo "film"

In [135]:
df_nan = pd.DataFrame(pd.isnull(df1.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df1) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

#print('Movie numpy:',movie_np)
#print('Length:',len(movie_np))

df = df1[pd.notnull(df1['Rating'])]

df['Movie_Id'] = movie_np.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)
print('-Dataset examples-')
print(df)


-Dataset examples-
      index  Cust_Id  Rating  Movie_Id
1         1  1488844     3.0         1
2         2   822109     5.0         1
3         3   885013     4.0         1
4         4    30878     4.0         1
5         5   823519     3.0         1
...     ...      ...     ...       ...
9995   9995  2142408     1.0         8
9996   9996  2231367     3.0         8
9997   9997  1304395     4.0         8
9998   9998  1468830     3.0         8
9999   9999  1369078     1.0         8

[9992 rows x 4 columns]


/tmp/ipykernel_9827/110950279.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Movie_Id'] = movie_np.astype(int)
/tmp/ipykernel_9827/110950279.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cust_Id'] = df['Cust_Id'].astype(int)


### Creo matrice di rating

In [136]:
ratings_matrix = df.pivot(index='Cust_Id', columns='Movie_Id', values='Rating')

# Se vuoi gestire eventuali valori mancanti con uno zero, puoi utilizzare il metodo fillna
ratings_matrix = ratings_matrix.fillna(0)

# Se preferisci avere la matrice come array numpy, puoi utilizzare values
#ratings_matrix_array = ratings_matrix.values

print(ratings_matrix)

Movie_Id    1    2    3    4    5    6    7    8
Cust_Id                                         
685       0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0
695       0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
915       5.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
967       0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
1333      0.0  0.0  4.0  0.0  0.0  0.0  0.0  3.0
...       ...  ...  ...  ...  ...  ...  ...  ...
2647871   4.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0
2648122   0.0  0.0  0.0  0.0  4.0  0.0  0.0  0.0
2648204   0.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0
2648650   0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0
2648861   0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0

[9619 rows x 8 columns]


### Prendo i primi y così che anche lo user based ha tanti film (un buon profilo ricco) ma riduzo il numero di utenti per non renderlo infinito

In [137]:
ratings_matrix = ratings_matrix.head(2000)

### rimuovo le righe nulle

In [138]:
print(ratings_matrix.shape)

(2000, 8)


In [76]:
ratings_matrix_support = pd.DataFrame()
for i in ratings_matrix.iterrows():
    val = [not(j) for j in i[1]]
    #print(i[1])
    #print(all(val))
    if not(all(val)):
        ratings_matrix_support = ratings_matrix_support.append(i[1])

/tmp/ipykernel_9827/3795530229.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings_matrix_support = ratings_matrix_support.append(i[1])
/tmp/ipykernel_9827/3795530229.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings_matrix_support = ratings_matrix_support.append(i[1])
/tmp/ipykernel_9827/3795530229.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings_matrix_support = ratings_matrix_support.append(i[1])
/tmp/ipykernel_9827/3795530229.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings_matrix_support = ratings_matrix_support.append(i[1])
/tmp/ipykernel_9827/3795530229.py:7: FutureWarning: The frame.append

KeyboardInterrupt: 

In [143]:
#ratings_matrix = ratings_matrix_support.copy()
print(ratings_matrix_support.shape)

(1000, 225)


# Creo Test set

### prendo il rating da colore che ne hanno almeno due e lo metto a 0

In [139]:
y_test = {}
count = 0
for i in ratings_matrix.iterrows():
    if count < 20:
        almenoUno = 0
        for j in range(1,len(i[1])):
            if i[1][j] != 0:
                if almenoUno:
                    y_test[(i[0],j)] = i[1][j]
                    i[1][j] = 0
                    count +=1
                    break
                else:
                    almenoUno= True
            

In [140]:
y_test

{(3321, 5): 4.0,
 (7798, 6): 3.0,
 (16272, 3): 4.0,
 (21722, 4): 3.0,
 (29948, 6): 3.0,
 (30878, 5): 1.0,
 (46086, 6): 3.0,
 (91245, 6): 3.0,
 (99400, 5): 5.0,
 (164845, 6): 1.0,
 (214166, 6): 2.0,
 (261764, 5): 1.0,
 (305344, 2): 1.0,
 (333358, 6): 3.0,
 (348960, 5): 4.0,
 (371701, 5): 2.0,
 (379184, 6): 5.0,
 (387418, 2): 1.0,
 (396595, 5): 5.0,
 (401047, 5): 1.0}

#### Calcolo rating medi degli utenti

In [144]:
rating_medi = {}
# calcolo rating medio senza considerare gli 0
for i in ratings_matrix.iterrows():
        # somma elementi != 0
        count = 0
        # quanti elementi
        countn = 0
        for j in i[1].values:
            # se non è 0
            if j != 0:
                  count += j
                  countn += 1
        if countn != 0:
                rating_medi[i[0]] = count/countn
        else:
                rating_medi[i[0]] = 0

print(rating_medi)

{685: 3.0, 695: 1.0, 915: 5.0, 967: 1.0, 1333: 3.5, 2133: 2.0, 2442: 3.0, 3292: 4.0, 3321: 3.0, 3417: 5.0, 3522: 2.0, 3718: 4.0, 3798: 4.0, 4016: 4.0, 4326: 4.0, 4456: 1.0, 4753: 5.0, 4783: 4.0, 5225: 4.0, 5749: 5.0, 6154: 4.0, 6303: 3.0, 6384: 3.0, 6460: 5.0, 6510: 3.0, 6689: 4.0, 7116: 1.0, 7284: 5.0, 7798: 4.0, 8012: 4.0, 8117: 5.0, 8121: 1.0, 8212: 4.0, 8753: 3.0, 8999: 2.0, 9660: 3.0, 10251: 4.0, 10612: 1.0, 10972: 3.0, 10996: 2.0, 11169: 2.0, 11186: 4.0, 11194: 3.0, 11409: 5.0, 11589: 3.0, 13432: 3.0, 13511: 5.0, 13561: 3.0, 13651: 3.0, 14035: 5.0, 14326: 4.0, 14389: 3.0, 14422: 2.0, 14728: 2.0, 14756: 4.0, 14924: 5.0, 15218: 5.0, 15288: 5.0, 15420: 2.0, 15796: 2.0, 16121: 1.0, 16229: 4.0, 16272: 3.0, 16818: 3.0, 17063: 5.0, 17088: 5.0, 17149: 1.0, 17510: 5.0, 17890: 3.0, 18753: 1.0, 18960: 2.0, 19006: 5.0, 19106: 2.0, 19289: 5.0, 20936: 1.0, 21153: 5.0, 21296: 4.0, 21368: 5.0, 21619: 2.0, 21722: 4.0, 21777: 3.0, 21983: 3.0, 22015: 4.0, 22178: 5.0, 22575: 5.0, 22663: 1.0, 22696: 

## Usare user based NN

### fun di similarità

In [106]:
# Definire la funzione per il calcolo della similarità tra utenti utilizzando la person coefficent
def user_similarity(ratings,user_target,id_user_target):
    similarità = {}
    # scorro UTENTI
    for i in ratings.iterrows():
        # se l'UTENTE è diverso dall'utente target
        if i[0] != id_user_target:
            # calcolo similarità
            numeratore = 0
            denominatoreP1 = 0
            denominatoreP2 = 0

            # scorro recensioni dell'utente
            for j in i[1].keys():
                numeratore += (i[1][j]-rating_medi[i[0]])*(user_target[j]-rating_medi[id_user_target])
                
                denominatoreP1 += (i[1][j]-rating_medi[i[0]])**2
                
                denominatoreP2 +=(user_target[j]-rating_medi[id_user_target])**2
            
            sim = 0
            if denominatoreP1 !=0 and denominatoreP2 != 0:
                sim = numeratore/(math.sqrt(denominatoreP1)*math.sqrt(denominatoreP2))
            
            similarità[(i[0],id_user_target)] = sim
    return similarità

### pre-processing sulla similarità

In [107]:
similarita_coppie = {}
for i,_ in y_test:
    # Calcolare la similarità per utenti del tewt set
    similarita_coppie.update(user_similarity(ratings_matrix,ratings_matrix.loc[i],i))

In [108]:
similarita_coppie

{(134, 7): 0.9655172413793104,
 (201, 7): 0.9655172413793104,
 (261, 7): 0.9655172413793104,
 (265, 7): 0.9655172413793105,
 (307, 7): 1.0000000000000002,
 (383, 7): 0.9655172413793105,
 (384, 7): 0.9655172413793104,
 (424, 7): 0.9655172413793104,
 (462, 7): 0.9655172413793104,
 (478, 7): 0.9655172413793104,
 (491, 7): 0.9655172413793105,
 (527, 7): 0.9655172413793104,
 (592, 7): 0.9655172413793104,
 (685, 7): 0.9655172413793105,
 (695, 7): 1.0,
 (734, 7): 1.0,
 (742, 7): 1.0,
 (815, 7): 0.9655172413793104,
 (834, 7): 0.9655172413793105,
 (840, 7): 0.9655172413793104,
 (906, 7): 0.9655172413793105,
 (911, 7): 0.9655172413793104,
 (915, 7): 0.9655172413793104,
 (933, 7): 0.9655172413793104,
 (967, 7): 0.9655172413793104,
 (979, 7): 0.9655172413793105,
 (1034, 7): 0.9655172413793104,
 (1086, 7): 0.9655172413793105,
 (1146, 7): 0.9655172413793105,
 (1186, 7): 0.9655172413793105,
 (1243, 7): 0.9655172413793104,
 (1310, 7): 0.9655172413793105,
 (1333, 7): 0.8865027457739031,
 (1353, 7): 0.9

In [84]:
utenteSimileUno = ratings_matrix.loc[261]
utenteSimileDue = ratings_matrix.loc[2044720]
print(utenteSimileUno)
print(utenteSimileDue)
print(user_target)


Movie_Id
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    4.0
Name: 261, dtype: float64
Movie_Id
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     1.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    1.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    4.0
Name: 2044720, dtype: float64
Movie_Id
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    3.0
Name: 2442, dtype

In [109]:
# FACCIO UNA SOGLIO K DI VICINATO SULLA BASE DELLA SIMILARITÀ
def calcoloSimilaritaConSoglia(similarita,id_user_target):
    sim_soglia = {}
    # scorro coppie di similarità (a,b)
    for i in similarita:
        # se la similarità rispetta la soglia
        if similarita[i] > 0.90:
            # verifico che sia del mio utente target
            if i[1] == id_user_target:
                sim_soglia[i] = similarita[i]
    return sim_soglia

#### fun Prediction

In [111]:
# Definire una funzione per la predizione delle valutazioni utilizzando User-Based Collaborative Filtering
def predict_user_based(item,user_similarities,ratings_matrix,rating_medi,id_user_target):
    # Calcolare le valutazioni previste
    num = 0
    den = 0
    pred_ratings = 0
    
    for i in user_similarities:     
        
        num += (user_similarities[i]* (ratings_matrix.loc[i[0]][item]-rating_medi[i[0]]))
        den += user_similarities[i]
    if den != 0:
        pred_ratings = rating_medi[id_user_target] + (num / den)
    else:
        pred_ratings = rating_medi[id_user_target]
    print(pred_ratings)
    return pred_ratings
    #else:
    #    return ratings_matrix.loc[id_user_target][item]

# Ottenere le previsioni per tutte le valutazioni
#predicted_ratings_user_based = predict_user_based(id_item_target, sim_soglia,ratings_matrix,rating_medi,id_user_target)
#print(f'Rating per l item {id_item_target} è di {predicted_ratings_user_based} per l utente {id_user_target}')


### testing model

In [112]:
# Ottenere le previsioni per tutte le valutazioni
y_pred = {}

for i in y_test:
    print(f'{i} starting...')

    # vettore rating utente target
    print(f'user target processing...')
    user_target = ratings_matrix.loc[i[0]]
    # SIMILARITÀ SOPRA CERTA SOGLIA
    print('sim_soglia processing...')
    sim_soglia = calcoloSimilaritaConSoglia(similarita_coppie,i[0])
    print(len(sim_soglia))
    # predico rating test
    print('predicting...')
    predicted_ratings_user_based = predict_user_based(i[1], sim_soglia, ratings_matrix,rating_medi,i[0])   
    if predicted_ratings_user_based < 0:
        predicted_ratings_user_based = 0
    # rating predetto
    print('Appending...')
    y_pred[i]=round(predicted_ratings_user_based,2)


(7, 28) starting...
user target processing...
sim_soglia processing...
993
predicting...
3.197949264765719
Appending...
(1333, 8) starting...
user target processing...
sim_soglia processing...
662
predicting...
-0.39541929277424126
Appending...
(1658, 26) starting...
user target processing...
sim_soglia processing...
994
predicting...
-1.3451470289568328
Appending...
(1811, 28) starting...
user target processing...
sim_soglia processing...
996
predicting...
0.19671153841257727
Appending...
(1871, 28) starting...
user target processing...
sim_soglia processing...
996
predicting...
2.198789374593523
Appending...
(2000, 28) starting...
user target processing...
sim_soglia processing...
996
predicting...
1.1977504565030501
Appending...
(2133, 17) starting...
user target processing...
sim_soglia processing...
994
predicting...
-0.7651863583439984
Appending...
(2213, 28) starting...
user target processing...
sim_soglia processing...
994
predicting...
1.2005304123339848
Appending...
(2264, 28

In [113]:
print(f'y_pred:{y_pred}')
print(f'y_test:{y_test}')

y_pred:{(7, 28): 3.2, (1333, 8): 0, (1658, 26): 0, (1811, 28): 0.2, (1871, 28): 2.2, (2000, 28): 1.2, (2133, 17): 0, (2213, 28): 1.2, (2264, 28): 2.2, (2976, 28): 1.2, (3321, 5): 0, (3518, 17): 0.74, (3718, 28): 2.2, (3998, 24): 0.98, (4326, 18): 0.98, (4477, 28): 1.2, (4706, 18): 1.97, (4906, 26): 0.15, (5225, 21): 0.47, (5652, 18): 0.46}
y_test:{(7, 28): 4.0, (1333, 8): 3.0, (1658, 26): 4.0, (1811, 28): 3.0, (1871, 28): 4.0, (2000, 28): 4.0, (2133, 17): 2.0, (2213, 28): 5.0, (2264, 28): 2.0, (2976, 28): 4.0, (3321, 5): 4.0, (3518, 17): 3.0, (3718, 28): 4.0, (3998, 24): 4.0, (4326, 18): 4.0, (4477, 28): 5.0, (4706, 18): 4.0, (4906, 26): 4.0, (5225, 21): 4.0, (5652, 18): 4.0}


### MAE, NMAE, RMSE, Precision, Recall, F1

In [118]:
# i'm deleting the rate predicted for evaluationg sim_soglia
""" for i in range(len(y_true)):rating_medi[id_user_target]
    if y_true[i] == 0:
        y_pred[i] = 0 """

# Calcola le metriche aggiuntive

precision = precision_score([round(y_test[pred]) for pred in y_test], [round(y_pred[pred]) for pred in y_pred], average='micro')
recall = recall_score([round(y_test[pred]) for pred in y_test], [round(y_pred[pred]) for pred in y_pred], average='micro')
f1 = f1_score([round(y_test[pred]) for pred in y_test], [round(y_pred[pred]) for pred in y_pred], average='micro')
#nmae = accuracy.mae(predictions) / (max(y_true) - min(y_true))
#mae = accuracy.mae(y_pred)

def mae(true,pred):
    n = len(true)
    tot_val = 0
    for i in true:
        val = abs(pred[i]-true[i])
        tot_val += val
    print(f'MAE:{(tot_val/n)}')

def Nmae(true,pred):
    n = len(true)
    tot_val = 0
    max = 0
    min = 9999
    for i in true:
        if true[i] < min:
            min = true[i]
        if true[i] > max:
            max = true[i]
        val = abs(pred[i]-true[i])
        tot_val += val
    print(f'NMAE:{(tot_val/(n*(max-min)))}')

def rmse(true,pred):
    n = len(true)
    tot_val = 0
    for i in true:
        val = (pred[i]-true[i])**2
        tot_val += val
    rapp = tot_val/n
    print(f'RMSE:{math.sqrt(rapp)}')

mae(y_test,y_pred)
Nmae(y_test,y_pred)
rmse(y_test,y_pred)

# Stampa i risultati
print(f'Precision:',precision)
print(f'Recall:',recall)
print(f'F1 Score:',f1)
#print(f'NMAE:',nmae)
#print(f'MAE:',mae)

MAE:2.7425
NMAE:0.9141666666666667
RMSE:2.9312480277178863
Precision: 0.05
Recall: 0.05
F1 Score: 0.05000000000000001


## Usare item based NN 

In [145]:
ratings_matrix_transpose = ratings_matrix.transpose()
#df.pivot(index='Movie_Id', columns='Cust_Id', values='Rating')

# Se vuoi gestire eventuali valori mancanti con uno zero, puoi utilizzare il metodo fillna
ratings_matrix_transpose = ratings_matrix_transpose.fillna(0)

# Se preferisci avere la matrice come array numpy, puoi utilizzare values
#ratings_matrix_array = ratings_matrix.values

print(ratings_matrix_transpose)

Cust_Id   685     695     915     967     1333    2133    2442    3292    \
Movie_Id                                                                   
1            0.0     0.0     5.0     0.0     0.0     0.0     3.0     0.0   
2            0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3            0.0     0.0     0.0     0.0     4.0     0.0     0.0     0.0   
4            0.0     0.0     0.0     1.0     0.0     0.0     0.0     0.0   
5            3.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
6            0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
7            0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
8            0.0     1.0     0.0     0.0     3.0     2.0     0.0     4.0   

Cust_Id   3321    3417    ...  545639  546338  546753  546933  547684  547732  \
Movie_Id                  ...                                                   
1            3.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     3.0

### fun adjusted cosine similarity

In [146]:
# Definire la funzione per il calcolo della similarità tra utenti utilizzando la person coefficent
def item_similarity(ratings,item_target,id_item_target):
    similarita_transpose = {}
    # scorro UTENTI
    for i in ratings.iterrows():
        # i riga film con rating utenti
        # se l'ITEM è diverso dall'item target
        if i[0] != id_item_target:
            # calcolo similarità
            numeratore = 0
            denominatoreP1 = 0
            denominatoreP2 = 0
            for j in i[1].keys():
                #print(j)
                # j utente
                #if i[1][j] != 0 and item_target[j] != 0:
                numeratore += (i[1][j]-rating_medi[j])*(item_target[j]-rating_medi[j])
                denominatoreP1 += (i[1][j]-rating_medi[j])**2
                denominatoreP2 +=(item_target[j]-rating_medi[j])**2
            sim = 0
            if denominatoreP1 != 0 and denominatoreP2 != 0:
                sim = numeratore/(math.sqrt(denominatoreP1)*math.sqrt(denominatoreP2))
            similarita_transpose[(i[0],id_item_target)] = sim

    return similarita_transpose
# Calcolare la similarità tra utenti
#item_similarity(ratings_matrix_transpose,item_target)
#print(similarita_transpose)

### Pre processing similarity

In [147]:
similarita_coppie_transpose = {}
for i,j in y_test:
    #print(i)
    # Calcolare la similarità tra utenti
    similarita_coppie_transpose.update(item_similarity(ratings_matrix_transpose,ratings_matrix_transpose.loc[j],j))

In [148]:
similarita_coppie_transpose

{(1, 5): 0.8712741992491929,
 (2, 5): 0.9069630209663478,
 (3, 5): 0.7585941953801564,
 (4, 5): 0.9127585913992952,
 (6, 5): 0.8620138786572321,
 (7, 5): 0.9159393596659783,
 (8, 5): 0.5902027376449837,
 (1, 6): 0.910835936819177,
 (2, 6): 0.9462552613181188,
 (3, 6): 0.8134361012838848,
 (4, 6): 0.950839714465675,
 (5, 6): 0.8620138786572321,
 (7, 6): 0.9537044795406581,
 (8, 6): 0.6680440623708519,
 (1, 3): 0.8204557510381436,
 (2, 3): 0.8629489205238129,
 (4, 3): 0.8696448740946174,
 (5, 3): 0.7585941953801564,
 (6, 3): 0.8134361012838848,
 (7, 3): 0.8729855191476875,
 (8, 3): 0.5069544666073028,
 (1, 4): 0.9560923412971506,
 (2, 4): 0.9866967541160616,
 (3, 4): 0.8696448740946174,
 (5, 4): 0.9127585913992952,
 (6, 4): 0.950839714465675,
 (7, 4): 0.9938184429609401,
 (8, 4): 0.7412931112346663,
 (1, 2): 0.9505967593090459,
 (3, 2): 0.8629489205238129,
 (4, 2): 0.9866967541160616,
 (5, 2): 0.9069630209663478,
 (6, 2): 0.9462552613181188,
 (7, 2): 0.9890951432826331,
 (8, 2): 0.731861

In [118]:
# FACCIO UNA SOGLIO K DI VICINATO SULLA BASE DELLA SIMILARITÀ
sim_soglia_transpose = {}
for i in similarita_transpose:
    if similarita_transpose[i] > 0.0:
        sim_soglia_transpose[i] = similarita_transpose[i]
print(sim_soglia_transpose)

{(2, 1): 0.045871176736922314}


### selecting item simili

In [149]:
# prendo solo gli item votati dall'utente target
def similaritaSoglia(similarita_transpose,id_item_target):
    sim_soglia_transpose = {}
    # scorro coppie similirarità
    for i in similarita_transpose:
        # controllo la soglia
        if similarita_transpose[i] > 0.70:
            # controllo se è la similarità del mio target
            if i[1] == id_item_target:
                sim_soglia_transpose[i] = similarita_transpose[i]
    return sim_soglia_transpose

### fun predict

In [150]:
# Definire una funzione per la predizione delle valutazioni utilizzando User-Based Collaborative Filtering
def predict_item_based(user,sim_soglia_transpose,ratings_matrix_transpose,id_item_target):
    # Calcolare le valutazioni previste
    num = 0
    den = 0
    #scorro item simili a tutti i target
    for i in sim_soglia_transpose:
        # controllo che sia simili al target passato
        if i[1] == id_item_target:
            # se l'utente l'ha votato
            if ratings_matrix_transpose.loc[i[0]][user] != 0:
                num += (sim_soglia_transpose[i]* (ratings_matrix_transpose.loc[i[0]][user]))
                den += sim_soglia_transpose[i]
    pred_ratings = 0
    if den != 0:
        pred_ratings = (num / den)
    return pred_ratings
    

# Ottenere le previsioni per tutte le valutazioni
#predicted_ratings_user_based = predict_item_based(id_item_target, similarita_transpose,ratings_matrix_transpose,id_item_target)
#print(f'Rating per l item {id_item_target} è di {predicted_ratings_user_based} per l utente {id_user_target}')


### Testing model

In [151]:
# Ottenere le previsioni per tutte le valutazioni
y_pred_transpose = {}

# scorro utenti target
for i in y_test:
    print(f'{i} starting...')
    # vettore rating utente target
    print(f'user target processing...')
    #item_target = ratings_matrix_transpose.loc[i[1]]
    
    print('sim_soglia processing...')
    # SIMILARITÀ SOPRA CERTA SOGLIA
    sim_soglia_transpose = similaritaSoglia(similarita_coppie_transpose,i[1])
    # predico rating test
    print('predicting...')
    predicted_ratings_item_based = predict_item_based(i[0], sim_soglia_transpose, ratings_matrix_transpose,i[1])   
    
    if predicted_ratings_item_based < 0:
        predicted_ratings_item_based = 0
    # rating predetto
    print('Appending...')
    y_pred_transpose[i]=round(predicted_ratings_item_based,2)


(3321, 5) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(7798, 6) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(16272, 3) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(21722, 4) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(29948, 6) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(30878, 5) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(46086, 6) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(91245, 6) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(99400, 5) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(164845, 6) starting...
user target processing...
sim_soglia processing...
predicting...
Appe

In [152]:
print(f'y_pred:{y_pred_transpose}')
print(f'y_true:{y_test}')

y_pred:{(3321, 5): 3.0, (7798, 6): 4.0, (16272, 3): 2.97, (21722, 4): 4.0, (29948, 6): 3.0, (30878, 5): 4.0, (46086, 6): 4.0, (91245, 6): 4.0, (99400, 5): 5.0, (164845, 6): 1.0, (214166, 6): 3.0, (261764, 5): 1.0, (305344, 2): 1.33, (333358, 6): 4.0, (348960, 5): 1.0, (371701, 5): 2.0, (379184, 6): 4.0, (387418, 2): 1.5, (396595, 5): 5.0, (401047, 5): 4.0}
y_true:{(3321, 5): 4.0, (7798, 6): 3.0, (16272, 3): 4.0, (21722, 4): 3.0, (29948, 6): 3.0, (30878, 5): 1.0, (46086, 6): 3.0, (91245, 6): 3.0, (99400, 5): 5.0, (164845, 6): 1.0, (214166, 6): 2.0, (261764, 5): 1.0, (305344, 2): 1.0, (333358, 6): 3.0, (348960, 5): 4.0, (371701, 5): 2.0, (379184, 6): 5.0, (387418, 2): 1.0, (396595, 5): 5.0, (401047, 5): 1.0}


### MAE, NMAE, RMSE, Precision, Recall, F1

In [153]:
# i'm deleting the rate predicted for evaluationg sim_soglia
""" for i in range(len(y_true)):
    if y_true[i] == 0:
        y_pred[i] = 0 """


precision = precision_score([round(y_test[pred]) for pred in y_test], [round(y_pred_transpose[pred]) for pred in y_pred_transpose], average='micro')
recall = recall_score([round(y_test[pred]) for pred in y_test], [round(y_pred_transpose[pred]) for pred in y_pred_transpose], average='micro')
f1 = f1_score([round(y_test[pred]) for pred in y_test], [round(y_pred_transpose[pred]) for pred in y_pred_transpose], average='micro')
#nmae = accuracy.mae(predictions) / (max(y_true) - min(y_true))
#mae = accuracy.mae(y_pred)

def mae(true,pred):
    n = len(true)
    tot_val = 0
    for i in pred:
        val = abs(pred[i]-true[i])
        tot_val += val
    print(f'MAE:{(tot_val/n)}')

def Nmae(true,pred):
    n = len(true)
    tot_val = 0
    max = 0
    min = 9999
    for i in pred:
        if true[i] < min:
            min = true[i]
        if true[i] > max:
            max = true[i]
        val = abs(pred[i]-true[i])
        tot_val += val
    print(f'NMAE:{(tot_val/(n*(max-min)))}')

def rmse(true,pred):
    n = len(true)
    tot_val = 0
    for i in pred:
        val = (pred[i]-true[i])**2
        tot_val += val
    rapp = tot_val/n
    print(f'RMSE:{math.sqrt(rapp)}')

mae(y_test,y_pred_transpose)
Nmae(y_test,y_pred_transpose)
rmse(y_test,y_pred_transpose)

# Stampa i risultati
print(f'Precision:',precision)
print(f'Recall:',recall)
print(f'F1 Score:',f1)
#print(f'NMAE:',nmae)
#print(f'MAE:',mae)

MAE:0.943
NMAE:0.23575
RMSE:1.3494406248516455
Precision: 0.35
Recall: 0.35
F1 Score: 0.35


## Paradigma : User based vs item base

## Matrix factorization: SVD 

In [154]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']][:], reader)
#data.split(n_folds=3)
trainsetData, testsetData = train_test_split(data, test_size=0.2)

In [155]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'],verbose=True,return_train_measures=True,n_jobs=100)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2346  1.2320  1.2553  1.2199  1.2108  1.2306  0.0151  
MAE (testset)     1.0027  1.0157  1.0382  0.9870  0.9843  1.0056  0.0199  
RMSE (trainset)   0.5410  0.5304  0.5181  0.5272  0.5184  0.5270  0.0085  
MAE (trainset)    0.4304  0.4195  0.4109  0.4213  0.4151  0.4195  0.0066  
Fit time          0.11    0.16    0.14    0.15    0.15    0.14    0.02    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


{'test_rmse': array([1.23459604, 1.23204715, 1.25534362, 1.21994966, 1.21081713]),
 'train_rmse': array([0.54104927, 0.53035118, 0.5181245 , 0.5272172 , 0.51838466]),
 'test_mae': array([1.00265054, 1.01572062, 1.0381757 , 0.98697732, 0.98433816]),
 'train_mae': array([0.43042986, 0.41952251, 0.41093438, 0.4213226 , 0.41512917]),
 'fit_time': (0.1127321720123291,
  0.1631627082824707,
  0.14147257804870605,
  0.14567780494689941,
  0.1497950553894043),
 'test_time': (0.011909961700439453,
  0.009044170379638672,
  0.008695125579833984,
  0.008764266967773438,
  0.009601116180419922)}

## NN

In [45]:
xTrain = pd.DataFrame({'Movie_Id': df['Movie_Id'], 'Cust_Id': df['Cust_Id']})
yTrain = pd.DataFrame({'Rating': df['Rating']})

print(xTrain.head())
print(yTrain.head())

   Movie_Id  Cust_Id
1         1  1488844
2         1   822109
3         1   885013
4         1    30878
5         1   823519
   Rating
1     3.0
2     5.0
3     4.0
4     4.0
5     3.0


In [47]:
# Input layer
ratings_input = Input(shape=(None,2))

# Embedding layers
dense0 = Dense(16, activation='relu')(ratings_input)

# Dense layers
#dense1 = Dense(128, activation='relu')(dense0)
dense2 = Dense(32, activation='relu')(dense0)

# Output layer
output = Dense(6,activation='softmax')(dense2)

# Model
model = Model(inputs=ratings_input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [52]:
model.fit(xTrain,yTrain, epochs=10, batch_size=64)

Epoch 1/10


16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 3/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 4/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 5/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 6/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 7/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 8/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 9/10
16/16 [==============================] - 0s 1ms/step - loss: 13.7193
Epoch 10/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193


In [35]:
y_pred = model.predict(xTrain)

32/32 [==============================] - 0s 1ms/step


In [40]:
print(y_pred)
y_pred_custom = []
for i in y_pred:
    val = -1
    index = -1
    for j in range(len(i)):
        if i[j] > val:
            val = i[j]
            index = j
    y_pred_custom.append(index)
y_pred_custom = pd.DataFrame(y_pred_custom)
print(y_pred_custom)
print(yTrain)


[[0.         0.         0.         0.         0.         0.99999994]
 [0.         0.         0.         0.         0.         0.99999994]
 [0.         0.         0.         0.         0.         0.99999994]
 ...
 [0.         0.         0.         0.         0.         1.        ]
 [0.         0.         0.         0.         0.         1.        ]
 [0.         0.         0.         0.         0.         1.        ]]
     0
0    5
1    5
2    5
3    5
4    5
..  ..
992  5
993  5
994  5
995  5
996  5

[997 rows x 1 columns]
     Rating
1       3.0
2       5.0
3       4.0
4       4.0
5       3.0
..      ...
995     3.0
996     4.0
997     4.0
998     3.0
999     4.0

[997 rows x 1 columns]


In [39]:
mae = np.mean(y_pred_custom - yTrain)
print(f'MAE: {mae}')

MAE: 0        NaN
Rating   NaN
dtype: float64


/usr/lib/python3/dist-packages/numpy/core/fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
